In [1]:
import os
import numpy as np
import rasterio
import matplotlib.image

from pathlib import Path


In [2]:
files = {}

for f in os.listdir("data"):
    if ".jp2" in f:
        parts = f.split("_")
        if parts[0] not in files:
            files[parts[0]] = {}
        if parts[1] not in files[parts[0]]:
            files[parts[0]][parts[1]] = {}
        files[parts[0]][parts[1]][parts[2]] = f"data/{f}"


In [3]:
dates = []
for f in files.values():
    dates += list(f.keys())
dates = sorted(list(set(dates)))
    

In [4]:
#dates = [d for d in dates if '2022' == d[:4] or '2023' == d[:4]]


In [5]:
s = 0
c = 4550

sj = 200

offsets = {
    'T34UDG': (30000+s, 0),
    'T34UEG': (30000+s, 10000),
    'T34UFG': (30000+s, 20000),
    'T34VDH': (20000+s, 0),
    'T34VDJ': (10000+s, 0),
    'T34VDK': (0+s, 0), # top left
    'T34VEH': (20000+s, 10000),
    'T34VEJ': (10000+s, 10000),
    'T34VEK': (0+s, 10000),
    'T34VFH': (20000+s, 20000),
    'T34VFJ': (10000+s, 20000),
    'T34VFK': (0+s, 20000),
    'T35ULB': (30000+sj, 30980-c),
    'T35UMB': (30000+sj, 40980-c),
    'T35UNB': (30000+sj, 50980-c),
    'T35VLC': (20000+sj, 30980-c),
    'T35VLD': (10000+sj, 30980-c),
    'T35VLE': (0+sj, 30980-c),
    'T35VMC': (20000+sj, 40980-c),
    'T35VMD': (10000+sj, 40980-c),
    'T35VME': (0+sj, 40980-c),
    'T35VNC': (20000+sj, 50980-c),
    'T35VND': (10000+sj, 50980-c),
    'T35VNE': (0+sj, 50980-c)
}


In [6]:
not_seen = {
    'T34UDG': 0,
    'T34UEG': 0,
    'T34UFG': 0,
    'T34VDH': 0,
    'T34VDJ': 0,
    'T34VDK': 0,
    'T34VEH': 0,
    'T34VEJ': 0,
    'T34VEK': 0,
    'T34VFH': 0,
    'T34VFJ': 0,
    'T34VFK': 0,
    'T35ULB': 0,
    'T35UMB': 0,
    #'T35UNB': 0,
    'T35VLC': 0,
    'T35VLD': 0,
    'T35VLE': 0,
    'T35VMC': 0,
    'T35VMD': 0,
    'T35VME': 0,
    #'T35VNC': 0,
    #'T35VND': 0,
    #'T35VNE': 0
}


In [7]:
files = {k:v for k,v in files.items() if k in not_seen}


In [8]:
rgb_composite = np.dstack(
    #(np.full((41680, 57410), 0.0), np.full((41680, 57410), 0.0), np.full((41680, 57410), 0.0))
    (np.full((41680, 47410), 0.0), np.full((41680, 47410), 0.0), np.full((41680, 47410), 0.0))
)


In [ ]:
gain = 2

for d in dates:
    print(d)
    for ki, vi in files.items():
        if d in vi:
            if ki in not_seen:
                del not_seen[ki]
            print("matched", ki, d)
            fj = vi[d]
            try:
                try:
                    #print("reading SCL", fj["SCL"])
                    scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                except BaseException as e:
                    if os.path.isfile(fj["SCL"]):
                        print("REMOVED", fj["SCL"])
                        os.remove(fj["SCL"])
                    raise e
                
                try:
                    #print("reading red", fj["B04"])
                    c_red = np.clip(rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
                except BaseException as e:
                    if os.path.isfile(fj["B04"]):
                        print("REMOVED", fj["B04"])
                        os.remove(fj["B04"])
                    raise e
                try:
                    #print("reading green", fj["B03"])
                    c_green = np.clip(rasterio.open(fj["B03"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)             
                except BaseException as e:
                    if os.path.isfile(fj["B03"]):
                        print("REMOVED", fj["B03"])
                        os.remove(fj["B03"])
                    raise e
                try:
                    #print("reading blue", fj["B02"])
                    c_blue = np.clip(rasterio.open(fj["B02"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)              
                except BaseException as e:
                    if os.path.isfile(fj["B02"]):
                        print("REMOVED", fj["B02"])
                        os.remove(fj["B02"])
                    raise e

                mask = ((scl != 0) & (scl != 1))
                #print(mask)
                #red[mask] = c_red[mask]
                #green[mask] = c_green[mask]
                #blue[mask] = c_blue[mask]
                #print("applying red")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 0][mask] = c_red[mask]
                #print("applying green")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 1][mask] = c_green[mask]
                #print("applying blue")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 2][mask] = c_green[mask]

            except BaseException as e:
                print(f"ERROR: {ki} {d} failed with {e}")
        else:
            print("no match", ki, d)
    #print("saving composite")
    if len(not_seen) > 0:
        print(f"missing cells, skipping save ({not_seen.keys()})")
    else:
        print("saving composite")
        matplotlib.image.imsave(f"{Path.home()}/Projs/bulbulis/notebooks/true_color_frames/{d}.jpeg", rgb_composite)


20150711T100006
no match T34VFH 20150711T100006
no match T34UDG 20150711T100006
no match T35UMB 20150711T100006
no match T34VDH 20150711T100006
no match T34VDJ 20150711T100006
no match T34VDK 20150711T100006
no match T34VEH 20150711T100006
no match T34UEG 20150711T100006
no match T35ULB 20150711T100006
matched T34VFJ 20150711T100006
no match T35VME 20150711T100006
no match T35VMC 20150711T100006
no match T35VMD 20150711T100006
no match T35VLD 20150711T100006
no match T34VEJ 20150711T100006
no match T34VFK 20150711T100006
no match T35VLC 20150711T100006
no match T35VLE 20150711T100006
no match T34VEK 20150711T100006
no match T34UFG 20150711T100006
missing cells, skipping save (dict_keys(['T34UDG', 'T34UEG', 'T34UFG', 'T34VDH', 'T34VDJ', 'T34VDK', 'T34VEH', 'T34VEJ', 'T34VEK', 'T34VFH', 'T34VFK', 'T35ULB', 'T35UMB', 'T35VLC', 'T35VLD', 'T35VLE', 'T35VMC', 'T35VMD', 'T35VME']))
20150724T101006
no match T34VFH 20150724T101006
no match T34UDG 20150724T101006
no match T35UMB 20150724T101006
